In [95]:
# We"ll need pandas for a dataframe
import pandas as pd
df = pd.DataFrame()
# And numpy
import numpy as np

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Create URL
import requests
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

# Get BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,"lxml")

# Test output
# print(soup.prettify())
# The table we need is of class "wikitable sortable"

My_table = soup.find("table",{"class":"wikitable sortable"})
# Preview extracted info
#print(My_table)
table_rows = My_table.findAll("tr")

#creating list of boroughs
l = []
for tr in table_rows:
    # The line below will potentially yield three fields: PostalCode, Borough and Neighborhood
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if (len(row)>1):
        # Check if there is the Borough isn't empty, skip line if yes
        if row[1] != 'Not assigned':
            row[2] = row[2].strip("\n")
            # If neighborhood is empty, use Borough
            if row[2] == 'Not assigned':
                row[2] = row[1]
            l.append(row)

# Check for duplicate mentions of PostalCode
# Create a "dupcheck" list, 1 if the PostalCode is a duplicate, 0 if not
# We are employing the fact that the list of Neighbourhoods and Boroughs are sorted by postalcode
dupcheck = []
for i in range(len(l)):
    if i != 0:
        if l[i-1][0] == l[i][0]:
            dupcheck.append(1)
        else:
            dupcheck.append(0)
    else:
        dupcheck.append(0)

# Use dupcheck list to make a correct list
# I am going to go over the list again and only append new PostalCodes
new_l = []
for i in range(len(l)):
    # This is a duplicate PostalCode, meaning we need to append to the existing neighbourhood list field
    if dupcheck[i] == 1:
        new_l[len(new_l)-1][2] += ", " + l[i][2] 
#        print("DUP, ", new_l[len(new_l)-1], len(new_l))
    else:
    # This is a new PostalCode, meaning we start a new neighbourhood list field
        new_l.append(l[i])
#        print("NEW, ", new_l[len(new_l)-1], len(new_l))

# Now we can create the desired dataframe
labels = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame.from_records(new_l, columns=labels)
df.shape

(103, 3)

In [2]:
!conda install -c conda-forge geocoder --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    certifi-2018.8.24          |        py36_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         427 KB

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0      conda-forge
    orderedset: 2.0-py36_0       conda-forge
    ratelim:    0.1.6-py36_0     conda-forge

The following packages will be UPDATED:

    certifi:    2018.8.24-py36_1 conda-forge --> 20

In [3]:
import geocoder # import geocoder
latitudes = []
longitudes = []
for postal_code in df['PostalCode']:
    # Looking for postal_code
    print("Looking for postal code ", postal_code)
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitudes.append(lat_lng_coords[0])
    longitudes.append(lat_lng_coords[1])
    
print(latitudes)

Looking for postal code  M3A
Looking for postal code  M4A
Looking for postal code  M5A
Looking for postal code  M6A
Looking for postal code  M7A
Looking for postal code  M9A
Looking for postal code  M1B
Looking for postal code  M3B
Looking for postal code  M4B
Looking for postal code  M5B
Looking for postal code  M6B
Looking for postal code  M9B
Looking for postal code  M1C
Looking for postal code  M3C
Looking for postal code  M4C
Looking for postal code  M5C
Looking for postal code  M6C
Looking for postal code  M9C
Looking for postal code  M1E
Looking for postal code  M4E
Looking for postal code  M5E
Looking for postal code  M6E
Looking for postal code  M1G
Looking for postal code  M4G
Looking for postal code  M5G
Looking for postal code  M6G
Looking for postal code  M1H
Looking for postal code  M2H
Looking for postal code  M3H
Looking for postal code  M4H
Looking for postal code  M5H
Looking for postal code  M6H
Looking for postal code  M1J
Looking for postal code  M2J
Looking for po

In [13]:
df['Latitude'] = pd.Series(latitudes, index=df.index)
df['Longitude'] = pd.Series(longitudes, index=df.index)

In [14]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Clustering part based on the Manhattan example lab

In [77]:
#print(df.shape)
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()
#print(toronto_data.shape)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [78]:
CLIENT_ID = 'QQIJ5Q05VJZQUMSMGZHVEPV3VI5RVUGGR1EXMM25WGI3WPMX' # your Foursquare ID
CLIENT_SECRET = '5GMK3PKCVT22NA5VDP5Y0N4ZHREEQDTRTQQ4EI50QO2XBM33' # your Foursquare Secret
VERSION = 20180901
LIMIT = 500

In [79]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [80]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [81]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business reply mail Processing Centre969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",52,52,52,52,52,52
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [82]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.00000,0.000000,0.00000,0.00000,0.010000,0.000000,0.000000,0.010000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business reply mail Processing Centre969 Eastern,0.062500,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,...,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.011765,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00000,0.011765,0.000000,0.000000,0.011765,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.01000,0.010000,0.00000,0.00000,0.060000,0.000000,0.040000,0.010000,0.00
8,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.011364,0.00,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011364,0.00000,0.000000,0.00000,0.00000,0.011364,0.011364,0.011364,0.000000,0.00


In [84]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.06
2              Bar  0.04
3  Thai Restaurant  0.04
4       Steakhouse  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2      Beer Bar  0.04
3   Cheese Shop  0.04
4    Steakhouse  0.04


----Brockton, Exhibition Place, Parkdale Village----
                  venue  freq
0           Coffee Shop  0.13
1             Nightclub  0.09
2                  Café  0.09
3        Breakfast Spot  0.09
4  Gym / Fitness Center  0.04


----Business reply mail Processing Centre969 Eastern----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2         Garden Center  0.06
3                Garden  0.06
4  Fast Food Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service 

Observe that some neighbourhoods above don't actually have five different types of venues in the neighbourhood.
However, as we are comparing relative frequencies when clustering different neighbourhoods, this should not skew the clustering by taking those nonexistent venues into account.

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,American Restaurant,Steakhouse,Thai Restaurant,Hotel,Restaurant,Gym,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Farmers Market,Restaurant,Café,Cheese Shop,Seafood Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Nightclub,Performing Arts Venue,Burrito Place,Caribbean Restaurant,Climbing Gym,Pet Store,Convenience Store
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Pizza Place,Farmers Market,Fast Food Restaurant,Spa,Brewery,Restaurant,Recording Studio,Auto Workshop,Burrito Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boutique
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Chinese Restaurant,Indian Restaurant,Pizza Place,Park,Italian Restaurant,Bakery,Pub
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Sandwich Place,Japanese Restaurant,Burger Joint,Bar,Bubble Tea Shop,Falafel Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Vietnamese Restaurant,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Hotpot Restaurant
8,Christie,Grocery Store,Café,Park,Nightclub,Convenience Store,Baby Store,Italian Restaurant,Restaurant,Diner,Athletics & Sports
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Gastropub,Café,Men's Store,Mediterranean Restaurant


In [59]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [61]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Pub,Mexican Restaurant,Health Food Store,Performing Arts Venue,Beer Store
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Bar,Japanese Restaurant,Italian Restaurant,Restaurant,American Restaurant,Plaza
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Clothing Store,Hotel,Cosmetics Shop,Gastropub,Italian Restaurant,Japanese Restaurant,Bakery
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Other Great Outdoors,Pub,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Farmers Market,Restaurant,Café,Cheese Shop,Seafood Restaurant


We're ready to create the map now, using Folium as in the Manhattan example.

In [68]:
# Toronto latitude and longitude
latitude = 43.6532
longitude = -79.3832

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront, Regent Park",0,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Pub,Mexican Restaurant,Health Food Store,Performing Arts Venue,Beer Store
1,"Ryerson, Garden District",0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Bar,Japanese Restaurant,Italian Restaurant,Restaurant,American Restaurant,Plaza
2,St. James Town,0,Coffee Shop,Restaurant,Café,Clothing Store,Hotel,Cosmetics Shop,Gastropub,Italian Restaurant,Japanese Restaurant,Bakery
3,The Beaches,0,Coffee Shop,Other Great Outdoors,Pub,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
4,Berczy Park,0,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Farmers Market,Restaurant,Café,Cheese Shop,Seafood Restaurant
5,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Sandwich Place,Japanese Restaurant,Burger Joint,Bar,Bubble Tea Shop,Falafel Restaurant
6,Christie,0,Grocery Store,Café,Park,Nightclub,Convenience Store,Baby Store,Italian Restaurant,Restaurant,Diner,Athletics & Sports
7,"Adelaide, King, Richmond",0,Coffee Shop,Café,Bar,American Restaurant,Steakhouse,Thai Restaurant,Hotel,Restaurant,Gym,Asian Restaurant
8,"Dovercourt Village, Dufferin",0,Supermarket,Bakery,Café,Fast Food Restaurant,Middle Eastern Restaurant,Discount Store,Bank,Bar,Brewery,Music Venue
9,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Scenic Lookout,Brewery,Italian Restaurant,Sports Bar,Sushi Restaurant


In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
# This cluster doesn't seem to make sense

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Lawrence Park,1,Park,Swim School,Bus Line,Gym / Fitness Center,College Rec Center,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
24,"Parkdale, Roncesvalles",1,Coffee Shop,Gift Shop,Restaurant,Eastern European Restaurant,Dessert Shop,Piano Bar,Burger Joint,Bar,Breakfast Spot,Cuban Restaurant
27,"Runnymede, Swansea",1,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Restaurant,Diner,Pub,Falafel Restaurant,Scenic Lookout


In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
# Trail, playground and park - only one of its kind?

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,"Moore Park, Summerhill East",2,Gym,Trail,Playground,Park,German Restaurant,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"Brockton, Exhibition Place, Parkdale Village",3,Coffee Shop,Breakfast Spot,Café,Nightclub,Performing Arts Venue,Burrito Place,Caribbean Restaurant,Climbing Gym,Pet Store,Convenience Store
35,"First Canadian Place, Underground city",3,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Deli / Bodega,Gastropub,Gym,Burger Joint


In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North Toronto West,4,Sporting Goods Shop,Coffee Shop,Yoga Studio,Fast Food Restaurant,Diner,Dessert Shop,Metro Station,Mexican Restaurant,Park,Chinese Restaurant


The above clusters don't seem to make sense, but I don't really know what went wrong.
Admittedly, 1) this kind of multi-dimensional clustering is hard to interpret by intuition and 
2) the clustering in the Manhattan example was non-intuitive as well.